In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

case = pd.read_csv("Data Resources/Case.csv")
patient = pd.read_csv("Data Resources/PatientInfo.csv")
policy = pd.read_csv("Data Resources/Policy.csv")
region = pd.read_csv("Data Resources/Region.csv")
search = pd.read_csv("Data Resources/SearchTrend.csv")
seoul = pd.read_csv("Data Resources/SeoulFloating.csv")
time = pd.read_csv("Data Resources/Time.csv")
timeAge = pd.read_csv("Data Resources/TimeAge.csv")
timeGender = pd.read_csv("Data Resources/TimeGender.csv")
timeProvince = pd.read_csv("t/TimeProvince.csv")
weather = pd.read_csv("../data/coronavirusdataset/Weather.csv")